In [78]:
import logging
from collections import deque
from pprint import pprint
import numpy as np
import time

logging.basicConfig(format='[%(asctime)s] %(levelname)s: %(message)s', datefmt='%H:%M:%S', level=logging.INFO)

In [79]:
def _contains_duplicates(X):
    return np.sum(np.unique(X)) != np.sum(X)

def contains_duplicates(sol):
    return any(_contains_duplicates(sol[r,:]) for r in range(9)) or \
           any(_contains_duplicates(sol[:,r]) for r in range(9)) or \
           any(_contains_duplicates(sol[r:r+3:,c:c+3]) for r in range(0,9,3) for c in range(0,9,3))

def valid_solution(sol):
    return not contains_duplicates(sol) and np.sum(sol) == (1+2+3+4+5+6+7+8+9) * 9

def print_sudoku(sudoku):
    print("+-------+-------+-------+")
    for b in range(0, 9, 3):
        for r in range(3):
            print("|", " | ".join(" ".join(str(_) for _ in sudoku[b+r, c:c+3]) for c in range(0, 9, 3)), "|")
        print("+-------+-------+-------+")

In [80]:
def dfsolve(sudoku):
    """Vanilla depth-first solver for sudoku puzzles"""
    frontier = deque([sudoku.copy()])
    num_nodes = 0
    while frontier:
        node = frontier.popleft()
        num_nodes += 1

        if valid_solution(node):
            logging.info(f"Solved after expanding {num_nodes:,} nodes")
            return node

        for i, j in zip(*np.where(node == 0)):
            for c in range(1, 10):
                node[i, j] = c
                if not contains_duplicates(node):
                    frontier.appendleft(node.copy())
    logging.info(f"Giving up after expanding {num_nodes:,} nodes")
    return None


In [81]:
simple_sudoku = np.array([[6, 0, 4,    0, 7, 0,    0, 0, 1],
                          [0, 5, 0,    0, 0, 0,    0, 7, 0],
                          [7, 0, 0,    5, 9, 6,    8, 3, 4],

                          [0, 8, 0,    0, 0, 2,    4, 9, 0],
                          [1, 0, 0,    0, 0, 0,    0, 0, 3],
                          [0, 6, 9,    7, 0, 0,    0, 5, 0],

                          [9, 1, 8,    3, 6, 7,    0, 0, 5],
                          [0, 4, 0,    0, 0, 0,    0, 6, 0],
                          [2, 0, 0,    0, 5, 0,    7, 0, 8]], dtype=np.int8)

hard_sudoku  =  np.array([[5, 8, 0,    0, 6, 0,    0, 3, 0],
                          [0, 0, 0,    7, 0, 0,    4, 0, 0],
                          [0, 0, 2,    0, 0, 0,    0, 0, 0],

                          [9, 3, 0,    0, 8, 0,    0, 5, 0],
                          [0, 0, 0,    0, 0, 6,    0, 0, 8],
                          [0, 1, 0,    0, 0, 0,    0, 0, 0],

                          [0, 0, 6,    0, 0, 0,    0, 2, 0],
                          [2, 9, 0,    0, 0, 4,    3, 0, 0],
                          [0, 0, 1,    0, 9, 0,    0, 0, 0]], dtype=np.int8)

In [82]:
def _valid_value(X):
    return np.sum(np.unique(X)) == np.sum(X)

def valid_value(node, i , j):
    a = (i//3) * 3
    b = (j//3) * 3
    return _valid_value(node[i,:]) and \
           _valid_value(node[:,j]) and \
           _valid_value(node[a:a+3:,b:b+3])

def _only_place_r(node, i, j, v):
    for x in zip(*np.where(node[i,:] == 0)):
        x = x[0]
        if x != j:
            node[i, x] = v
            if valid_value(node, i, x):
                node[i, x] = 0
                return False
            else:
                node[i, x] = 0
    return True

def _only_place_c(node, i, j, v):
    for y in zip(*np.where(node[:,j] == 0)):
        y = y[0]
        if y != i:
            node[y, j] = v
            if valid_value(node, y, j):
                node[y, j] = 0
                return False
            else:
                node[y, j] = 0
    return True

def _only_place_q(node, i, j, v):
    a = (i//3) * 3
    b = (j//3) * 3
    for x, y in zip(*np.where(node[a:a+3,b:b+3] == 0)):
        if a+x != i or b+y != j:
            node[a+x, b+y] = v
            if valid_value(node, a+x, b+y):
                node[a+x, b+y] = 0
                return False
            else:
                node[a+x, b+y] = 0
    return True


def only_place(node, i, j, v):

    return _only_place_r(node, i, j, v) or _only_place_c(node, i, j, v) or _only_place_q(node, i, j, v)

def sorted_values(node):
    vec = []
    for i, j in zip(*np.where(node == 0)):
        possible_values = []
        for v in range(1, 10):
            node[i, j] = v
            if valid_value(node, i, j):
                possible_values.append(v)
        node[i, j] = 0
        l = len(possible_values)
        if l > 0:
            vec.append(((i, j), (l, possible_values)))
    vec.sort(key= lambda x:x[1][0])
    return vec


def reliable_value(node, i, j):
    possible_values = []
    for v in range(1,10):
        node[i, j] = v
        if valid_value(node, i, j):
            possible_values.append(v)
    node[i, j] = 0
    if len(possible_values) == 1:
        return possible_values[0]
    else:
        for value in possible_values:
            if only_place(node, i, j, value):
                return value

    return False

def solve_sudoku(sudoku):
    frontier = deque([sudoku.copy()])
    num_nodes = 0
    while frontier:
        node = frontier.popleft()
        num_nodes += 1
        if num_nodes >= 5000:
            break

        changed = True
        while changed:
            changed = False
            for i, j in zip(*np.where(node == 0)):
                v = reliable_value(node, i, j)
                if v:
                    node[i, j] = v
                    changed = True

        if valid_solution(node):
            logging.info(f"Solved after expanding {num_nodes:,} nodes")
            return node

        # sv = sorted_values(node)
        # for (i, j), (_, x) in sv:
        #     if contains_duplicates(node):
        #         break
        #     for v in range(1,10):
        #         node[i, j] = v
        #         if valid_value(node, i, j):
        #             frontier.appendleft(node.copy())

        for i, j in zip(*np.where(node == 0)):
            if contains_duplicates(node):
                break
            for c in range(1, 10):
                node[i, j] = c
                if valid_value(node, i, j):
                    frontier.appendleft(node.copy())

    logging.info(f"Giving up after expanding {num_nodes:,} nodes")
    return None


In [83]:
def sudoku_generator(sudokus=1, *, kappa=5, random_seed=None):
    if random_seed:
        np.random.seed(random_seed)
    for puzzle in range(sudokus):
        sudoku = np.zeros((9, 9), dtype=np.int8)
        for cell in range(np.random.randint(kappa)):
            for p, val in zip(np.random.randint(0, 8, size=(9, 2)), range(1, 10)):
                tmp = sudoku.copy()
                sudoku[tuple(p)] = val
                if contains_duplicates(sudoku):
                    sudoku = tmp
        yield sudoku.copy()


In [84]:
for sudoku in sudoku_generator(random_seed=34):
    print_sudoku(sudoku)
    start_time = time.time()
    solution = solve_sudoku(sudoku)
    if solution is not None:
        print_sudoku(solution)
    print("Solution1 found in ", time.time()-start_time, "seconds")
    # start_time2 = time.time()
    # solution2 = dfsolve(sudoku)
    # if solution2 is not None:
    #     print("Solution2 found in ", time.time()-start_time2, "seconds")
    #     print_sudoku(solution2)

+-------+-------+-------+
| 0 0 0 | 0 0 0 | 0 0 0 |
| 8 0 0 | 0 0 0 | 2 0 0 |
| 0 0 1 | 0 0 7 | 0 0 0 |
+-------+-------+-------+
| 0 0 5 | 0 0 4 | 0 0 0 |
| 0 0 0 | 0 0 0 | 0 0 0 |
| 0 0 0 | 0 3 0 | 9 0 0 |
+-------+-------+-------+
| 0 0 0 | 0 0 6 | 0 0 0 |
| 0 0 0 | 0 0 0 | 0 0 0 |
| 0 0 0 | 0 0 0 | 0 0 0 |
+-------+-------+-------+


[11:05:02] INFO: Solved after expanding 30 nodes


+-------+-------+-------+
| 9 7 6 | 8 5 2 | 4 3 1 |
| 8 5 4 | 9 1 3 | 2 7 6 |
| 3 2 1 | 6 4 7 | 8 9 5 |
+-------+-------+-------+
| 7 9 5 | 2 8 4 | 6 1 3 |
| 4 8 3 | 1 6 9 | 7 5 2 |
| 6 1 2 | 7 3 5 | 9 8 4 |
+-------+-------+-------+
| 5 4 9 | 3 7 6 | 1 2 8 |
| 2 6 8 | 5 9 1 | 3 4 7 |
| 1 3 7 | 4 2 8 | 5 6 9 |
+-------+-------+-------+
Solution1 found in  3.687411069869995 seconds
